In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import glob
import os
import PIL
import PIL.Image
import imageio
import imgaug as ia
import random

from pathlib import Path
from numpy import asarray
from imgaug import augmenters as iaa
from tensorflow import keras
from keras import backend as K
from keras import layers
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from skimage.io import imread
from skimage import exposure, color
from skimage.transform import resize

In [2]:
train_data_path = '../../sensix_daninhas/dataset_100x100/train'
test_data_path  = '../../sensix_daninhas/dataset_100x100/validation'

img_rows = 100
img_cols = 100
epochs = 500
batch_size = 2
n_channels = 3

In [3]:
num_of_train_samples = len(glob.glob(train_data_path + "/**/*" + ".jpg" , recursive=True))
num_of_test_samples  = len(glob.glob(test_data_path + "/**/*" + ".jpg"  , recursive=True))
print("Train Images : ", num_of_train_samples)
print("Test Images  : ", num_of_test_samples)

Train Images :  199
Test Images  :  49


In [4]:
def random_func(*, percentage: int) -> int:
    '''
    percentage: Valores entre 0 e 100. Caso percentage > _value é retornado true, caso não
    é retornado false
    
    description: Utilizada para definir se a função de pré processamento será utilizada ou
    não. 
    
    return:
    bool
    '''
    _value = random.randint(0, 100)
    if _value < percentage:
        #print(_value)
        return False
    else:
        #print(_value)
        return True
    
#print(random_func(percentage = 50))

In [5]:
#def random_func():
#    _value = random.randint(0, 100)
#    return _value
#    
#print(random_func())

def prepare_dataset(img):
    '''
    img: array que representa a imagem que será modifiada:
    
    description: Utilizada para definir qual função de pré processamento será utilizada,
    cada intervalo representa um tipo de função a ser chamada. 
    [0  10] - imagem não transformada 
    [11 39] - prepare_dataset_func_1(img)
    [40 69] - prepare_dataset_func_2(img)
    [70 ..] - prepare_dataset_func_3(img)
    
    return: array
    O array retornado representa a imagem modificada ou não
    '''
    _choose_func = random_func()
    if _choose_func < 10:
        #print('fun 0')
        return img
    elif _choose_func  >= 10 and _choose_func < 40 :
        _img = prepare_dataset_func_1(img)
        #print('fun 1')
        return _img
    elif _choose_func  >= 40 and _choose_func < 70 :
        _img = prepare_dataset_func_2(img)
        #print('fun 2')
        return _img
    else:
        _img = prepare_dataset_func_3(img)
        #print('fun 3')
        return _img

In [6]:
def prepare_dataset_func_1(img):
    '''
    img: array que representa a imagem que será modificada:
    
    description: Representa efeitos relacionados a coloração da imagem. Nenhuma
    rotação ou aproximação (zoom) é dado. Apenas possui filtros que alteram as 
    caracteristicas gerais das imagens
    
    return: array
    O array retornado representa a imagem modificada
    '''
    img = img.astype('uint8')
    if random_func(percentage = 10):
        seq = iaa.Sequential([
            #iaa.Affine(rotate=(0, 90)),
            #iaa.Crop(percent=(0.1, 0.2)),
            iaa.LinearContrast((1.0, 1.9)),
            iaa.Multiply((0.8, 1.2), per_channel=0.2),
            iaa.AdditiveGaussianNoise(scale=(10, 20)),
            #iaa.Crop(percent=(0, 0.2)),
            iaa.AddToBrightness((-30, 30)),
            iaa.AddToHue((-50, 50)),
            #iaa.AllChannelsHistogramEqualization()
        ],
        random_order=True)
        images_aug = seq(image=img)
    else:
        images_aug = img
    
    images_aug = np.expand_dims(images_aug, axis=0)
    images_aug = images_aug.astype('float32')
    return (images_aug)

In [7]:
def prepare_dataset_func_2(img):
    '''
    img: array que representa a imagem que será modifiada:
    
    description: Representa efeitos relacionados a coloração da imagem, porém é 
    uma versão reduzida da prepare_dataset_func_1, aplicando apenas alterações
    nas cores dos canais RGB e modificações leves de contraste.
    
    return: array
    O array retornado representa a imagem modificada
    '''
    img = img.astype('uint8')
    seq = iaa.Sequential([
        iaa.WithChannels(2, [
            iaa.Multiply((0.5, 1.0)),
            iaa.LinearContrast((0.75, 1.25))
        ])
        
    ],
    random_order=True)

    images_aug = seq(image=img)
    images_aug = np.expand_dims(images_aug, axis=0)
    images_aug = images_aug.astype('float32')
    return (images_aug)

In [8]:
def prepare_dataset_func_3(img):
    '''
    img: array que representa a imagem que será modifiada:
    
    description: Representa efeitos relacionados as dimensões das imagens.
    São aplicadas rotações, aproximações (zoom), alteração aleátoria da cor
    do fundo da imagem, translação e rotação de imagens.
    
    return: array
    O array retornado representa a imagem modificada
    '''
    img = img.astype('uint8')
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)
    seq = iaa.Sequential([
        sometimes(iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-45, 45),
        shear=(-16, 16),
        order=[0, 1],
        cval=(0, 255),
        mode=ia.ALL))
    ],
    random_order=True)

    images_aug = seq(image=img)
    images_aug = np.expand_dims(images_aug, axis=0)
    images_aug = images_aug.astype('float32')
    return (images_aug)

In [9]:
def prepare_dataset_func_4(img):
    images_aug = vari_func(img, ch1 = 'green', ch2 = 'red', ch3 = 'blue', v_center=2)
    return (images_aug)

In [10]:
from PIL import Image  
from matplotlib import colors
import PIL  
from numpy import asarray

import matplotlib.pyplot as plt
import numpy as np
import cv2

def vari_func(img, ch1 = 'red', ch2 = 'green', ch3 = 'blue', v_min = 0, v_max = 10, v_center = 5, plot = False):
    '''
    img : array que representa a imagem que será modifiada:
    ch1, ch2, ch3 : canais utilizagos na imagem : Ex: ch1 = red, ch2 = green
    v_min : Representa os intervalos máximos e mínimos, apenas necessário na
    visualização do plot da imagem, que é quando plot é true
    plot : Necessário para plotar ou não a imagem modificada com o seu VARI
    
    description: Efetua o calcuo do (VARI), Índice Resistente à Atmosfera na 
    Região Visível e retorna a imagem modificada pelo VARI.
    
    return: array
    O array retornado representa a imagem modificada
    '''
    _r,_g,_b = cv2.split(img)
    _vari = (_g - _r) / ( _g + _r - _b)
    _vari = np.stack((_vari,)*n_channels, axis=-1)
    if plot:
        _cmap = colors.LinearSegmentedColormap.from_list('', [ch1, ch2, ch3])
        _norm = colors.DivergingNorm(vmin=v_min, vcenter=v_center, vmax=v_max)
        _n_img = plt.imshow(_vari.astype(np.float64), cmap=_cmap, norm=_norm)
        plt.axis('off')
    return _vari

In [11]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   preprocessing_function=prepare_dataset_func_1,
                                   fill_mode='constant',
                                   #horizontal_flip=True,
                                   #vertical_flip=True,
                                   )

test_datagen = ImageDataGenerator(rescale=1. / 255,
                                  #preprocessing_function=prepare_dataset_func_1,
                                  #fill_mode='constant',
                                  #horizontal_flip=True,
                                  #vertical_flip=True,
                                  )

train_generator = train_datagen.flow_from_directory(train_data_path,
                                                    target_size=(img_rows, img_cols),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    #save_to_dir='genereted_train',
                                                    #save_prefix = 'train'
                                                    )

validation_generator = test_datagen.flow_from_directory(test_data_path,
                                                        target_size=(img_rows, img_cols),
                                                        batch_size=batch_size,
                                                        class_mode='categorical',
                                                        #save_to_dir='genereted_val',
                                                        #save_prefix = 'val-'
                                                       )

Found 199 images belonging to 2 classes.
Found 49 images belonging to 2 classes.


In [12]:
model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(img_rows, img_cols, n_channels), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(32, (3, 3), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(64, (3, 3), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=100, 
        verbose=1, mode='auto', restore_best_weights=True)

history = model.fit(train_generator,
                    steps_per_epoch=num_of_train_samples // batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=num_of_test_samples // batch_size,
                    #callbacks=[monitor],verbose=2 # interrompe quando o overfiting começa
                    )

Y_pred_model = model.predict(validation_generator, num_of_test_samples // batch_size)
y_pred_model = np.argmax(Y_pred_model, axis=1)
print("Real :       ", validation_generator.classes)
print("Prediction : ", y_pred_model)
print('Confusion ValueError: could not broadcast input array from shape (1,100,100) into shape (100,100,3)Matrix')
print(confusion_matrix(validation_generator.classes, y_pred_model))

Epoch 1/500
99/99 [==============================] - 4s 29ms/step - loss: 0.7776 - accuracy: 0.5454 - val_loss: 0.8185 - val_accuracy: 0.4792
Epoch 2/500
99/99 [==============================] - 2s 25ms/step - loss: 0.7518 - accuracy: 0.5103 - val_loss: 0.7124 - val_accuracy: 0.5000
Epoch 3/500
99/99 [==============================] - 2s 25ms/step - loss: 0.7174 - accuracy: 0.5404 - val_loss: 0.6977 - val_accuracy: 0.4792
Epoch 4/500
99/99 [==============================] - 2s 24ms/step - loss: 0.7249 - accuracy: 0.4867 - val_loss: 0.6926 - val_accuracy: 0.5208
Epoch 5/500
99/99 [==============================] - 2s 24ms/step - loss: 0.7233 - accuracy: 0.5615 - val_loss: 0.6950 - val_accuracy: 0.4792
Epoch 6/500
99/99 [==============================] - 2s 25ms/step - loss: 0.6956 - accuracy: 0.5326 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 7/500
99/99 [==============================] - 2s 25ms/step - loss: 0.7121 - accuracy: 0.5659 - val_loss: 0.6929 - val_accuracy: 0.5000
Epoch 

99/99 [==============================] - 2s 24ms/step - loss: 0.2774 - accuracy: 0.9030 - val_loss: 0.6072 - val_accuracy: 0.8125
Epoch 59/500
99/99 [==============================] - 2s 23ms/step - loss: 0.2339 - accuracy: 0.8810 - val_loss: 0.4440 - val_accuracy: 0.8333
Epoch 60/500
99/99 [==============================] - 2s 24ms/step - loss: 0.2799 - accuracy: 0.9061 - val_loss: 0.7726 - val_accuracy: 0.7708
Epoch 61/500
99/99 [==============================] - 2s 24ms/step - loss: 0.4736 - accuracy: 0.8750 - val_loss: 0.5168 - val_accuracy: 0.8542
Epoch 62/500
99/99 [==============================] - 2s 24ms/step - loss: 0.3303 - accuracy: 0.9623 - val_loss: 0.7099 - val_accuracy: 0.7917
Epoch 63/500
99/99 [==============================] - 2s 24ms/step - loss: 0.1834 - accuracy: 0.9384 - val_loss: 0.4257 - val_accuracy: 0.8958
Epoch 64/500
99/99 [==============================] - 2s 24ms/step - loss: 0.1612 - accuracy: 0.9574 - val_loss: 0.5608 - val_accuracy: 0.8125
Epoch 65/500

99/99 [==============================] - 2s 24ms/step - loss: 0.0235 - accuracy: 0.9954 - val_loss: 0.5034 - val_accuracy: 0.8958
Epoch 116/500
99/99 [==============================] - 2s 24ms/step - loss: 0.5131 - accuracy: 0.8978 - val_loss: 0.5912 - val_accuracy: 0.8125
Epoch 117/500
99/99 [==============================] - 2s 24ms/step - loss: 0.0922 - accuracy: 0.9852 - val_loss: 1.0351 - val_accuracy: 0.8333
Epoch 118/500
99/99 [==============================] - 2s 24ms/step - loss: 0.2213 - accuracy: 0.9763 - val_loss: 0.7554 - val_accuracy: 0.7917
Epoch 119/500
99/99 [==============================] - 2s 24ms/step - loss: 0.0637 - accuracy: 0.9837 - val_loss: 1.1010 - val_accuracy: 0.7917
Epoch 120/500
99/99 [==============================] - 2s 25ms/step - loss: 0.0072 - accuracy: 1.0000 - val_loss: 1.2430 - val_accuracy: 0.8333
Epoch 121/500
99/99 [==============================] - 2s 24ms/step - loss: 0.0359 - accuracy: 0.9875 - val_loss: 1.8489 - val_accuracy: 0.7708
Epoch 

Epoch 172/500
99/99 [==============================] - 2s 23ms/step - loss: 0.2413 - accuracy: 0.9664 - val_loss: 1.5958 - val_accuracy: 0.8333
Epoch 173/500
99/99 [==============================] - 2s 24ms/step - loss: 0.5915 - accuracy: 0.9092 - val_loss: 1.2641 - val_accuracy: 0.8750
Epoch 174/500
99/99 [==============================] - 2s 24ms/step - loss: 0.0594 - accuracy: 0.9930 - val_loss: 0.6513 - val_accuracy: 0.8125
Epoch 175/500
99/99 [==============================] - 2s 24ms/step - loss: 0.1856 - accuracy: 0.9813 - val_loss: 1.3700 - val_accuracy: 0.8125
Epoch 176/500
99/99 [==============================] - 2s 24ms/step - loss: 0.0946 - accuracy: 0.9693 - val_loss: 1.2999 - val_accuracy: 0.8542
Epoch 177/500
99/99 [==============================] - 2s 23ms/step - loss: 0.0658 - accuracy: 0.9727 - val_loss: 1.8084 - val_accuracy: 0.8333
Epoch 178/500
99/99 [==============================] - 2s 25ms/step - loss: 0.1625 - accuracy: 0.9718 - val_loss: 1.1887 - val_accuracy:

Epoch 229/500
99/99 [==============================] - 2s 25ms/step - loss: 0.0252 - accuracy: 0.9820 - val_loss: 1.7211 - val_accuracy: 0.8125
Epoch 230/500
99/99 [==============================] - 2s 24ms/step - loss: 0.0887 - accuracy: 0.9903 - val_loss: 1.5784 - val_accuracy: 0.8125
Epoch 231/500
99/99 [==============================] - 2s 24ms/step - loss: 0.2418 - accuracy: 0.9817 - val_loss: 2.2216 - val_accuracy: 0.7917
Epoch 232/500
99/99 [==============================] - 2s 24ms/step - loss: 0.0183 - accuracy: 0.9984 - val_loss: 1.4192 - val_accuracy: 0.8542
Epoch 233/500
99/99 [==============================] - 2s 24ms/step - loss: 0.3397 - accuracy: 0.9877 - val_loss: 1.8958 - val_accuracy: 0.7917
Epoch 234/500
99/99 [==============================] - 2s 24ms/step - loss: 0.0351 - accuracy: 0.9951 - val_loss: 2.7767 - val_accuracy: 0.8125
Epoch 235/500
99/99 [==============================] - 2s 24ms/step - loss: 0.0577 - accuracy: 0.9910 - val_loss: 1.8037 - val_accuracy:

Epoch 286/500
99/99 [==============================] - 2s 24ms/step - loss: 0.1520 - accuracy: 0.9803 - val_loss: 1.4576 - val_accuracy: 0.7917
Epoch 287/500
99/99 [==============================] - 2s 24ms/step - loss: 0.0163 - accuracy: 0.9895 - val_loss: 1.1731 - val_accuracy: 0.8125
Epoch 288/500
99/99 [==============================] - 2s 24ms/step - loss: 0.0165 - accuracy: 0.9967 - val_loss: 0.8156 - val_accuracy: 0.8333
Epoch 289/500
99/99 [==============================] - 2s 24ms/step - loss: 0.0721 - accuracy: 0.9808 - val_loss: 0.5101 - val_accuracy: 0.8958
Epoch 290/500
99/99 [==============================] - 2s 25ms/step - loss: 0.0165 - accuracy: 0.9943 - val_loss: 1.9556 - val_accuracy: 0.8125
Epoch 291/500
99/99 [==============================] - 2s 24ms/step - loss: 0.0084 - accuracy: 0.9995 - val_loss: 2.0350 - val_accuracy: 0.8542
Epoch 292/500
99/99 [==============================] - 2s 24ms/step - loss: 0.1322 - accuracy: 0.9926 - val_loss: 1.4052 - val_accuracy:

Epoch 343/500
99/99 [==============================] - 2s 24ms/step - loss: 0.0167 - accuracy: 0.9929 - val_loss: 2.9409 - val_accuracy: 0.7917
Epoch 344/500
99/99 [==============================] - 2s 25ms/step - loss: 0.0936 - accuracy: 0.9924 - val_loss: 1.7582 - val_accuracy: 0.8542
Epoch 345/500
99/99 [==============================] - 2s 24ms/step - loss: 0.0024 - accuracy: 0.9990 - val_loss: 2.6915 - val_accuracy: 0.8750
Epoch 346/500
99/99 [==============================] - 2s 24ms/step - loss: 0.0515 - accuracy: 0.9986 - val_loss: 0.8080 - val_accuracy: 0.8125
Epoch 347/500
99/99 [==============================] - 2s 24ms/step - loss: 0.1073 - accuracy: 0.9654 - val_loss: 3.3190 - val_accuracy: 0.8125
Epoch 348/500
99/99 [==============================] - 2s 24ms/step - loss: 0.2225 - accuracy: 0.9875 - val_loss: 1.5228 - val_accuracy: 0.8542
Epoch 349/500
99/99 [==============================] - 2s 24ms/step - loss: 0.0055 - accuracy: 0.9979 - val_loss: 2.5468 - val_accuracy:

Epoch 400/500
99/99 [==============================] - 2s 24ms/step - loss: 0.0500 - accuracy: 0.9908 - val_loss: 1.9979 - val_accuracy: 0.7708
Epoch 401/500
99/99 [==============================] - 2s 24ms/step - loss: 0.0721 - accuracy: 0.9858 - val_loss: 2.1007 - val_accuracy: 0.8333
Epoch 402/500
99/99 [==============================] - 2s 24ms/step - loss: 0.0195 - accuracy: 0.9958 - val_loss: 2.3395 - val_accuracy: 0.7500
Epoch 403/500
99/99 [==============================] - 2s 24ms/step - loss: 0.1904 - accuracy: 0.9852 - val_loss: 2.8812 - val_accuracy: 0.8125
Epoch 404/500
99/99 [==============================] - 2s 24ms/step - loss: 0.1045 - accuracy: 0.9897 - val_loss: 2.8891 - val_accuracy: 0.8333
Epoch 405/500
99/99 [==============================] - 2s 24ms/step - loss: 0.4761 - accuracy: 0.9637 - val_loss: 2.8379 - val_accuracy: 0.7917
Epoch 406/500
99/99 [==============================] - 2s 24ms/step - loss: 0.0607 - accuracy: 0.9965 - val_loss: 3.3313 - val_accuracy:

99/99 [==============================] - 2s 25ms/step - loss: 0.0568 - accuracy: 0.9825 - val_loss: 2.1312 - val_accuracy: 0.8750
Epoch 457/500
99/99 [==============================] - 2s 24ms/step - loss: 0.5196 - accuracy: 0.9455 - val_loss: 3.4463 - val_accuracy: 0.7917
Epoch 458/500
99/99 [==============================] - 2s 24ms/step - loss: 0.5722 - accuracy: 0.9827 - val_loss: 3.0883 - val_accuracy: 0.7917
Epoch 459/500
99/99 [==============================] - 2s 24ms/step - loss: 1.4008e-04 - accuracy: 1.0000 - val_loss: 4.6274 - val_accuracy: 0.7500
Epoch 460/500
99/99 [==============================] - 2s 24ms/step - loss: 0.2632 - accuracy: 0.9792 - val_loss: 5.0790 - val_accuracy: 0.8125
Epoch 461/500
99/99 [==============================] - 2s 25ms/step - loss: 6.0846e-05 - accuracy: 1.0000 - val_loss: 5.3958 - val_accuracy: 0.8125
Epoch 462/500
99/99 [==============================] - 2s 24ms/step - loss: 0.0381 - accuracy: 0.9943 - val_loss: 3.8134 - val_accuracy: 0.833

In [13]:
model_reduced = Sequential()
model_reduced.add(Convolution2D(32, (3, 3), input_shape=(img_rows, img_cols, n_channels), padding='valid'))
model_reduced.add(Activation('relu'))
model_reduced.add(MaxPooling2D(pool_size=(2, 2)))
model_reduced.add(Convolution2D(64, (3, 3), padding='valid'))
model_reduced.add(Activation('relu'))
model_reduced.add(MaxPooling2D(pool_size=(2, 2)))
model_reduced.add(Flatten())
model_reduced.add(Dense(64))
model_reduced.add(Activation('relu'))
model_reduced.add(Dropout(0.5))
model_reduced.add(Dense(2))
model_reduced.add(Activation('sigmoid'))

model_reduced.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=100, 
        verbose=1, mode='auto', restore_best_weights=True)

history_model_reduced = model_reduced.fit(train_generator,
                    steps_per_epoch=num_of_train_samples // batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=num_of_test_samples // batch_size,
                    #callbacks=[monitor],verbose=2 # interrompe quando o overfiting começa
                    )

Y_pred_model_reduced = model_reduced.predict(validation_generator, num_of_test_samples // batch_size)
y_pred_model_reduced = np.argmax(Y_pred_model_reduced, axis=1)
print("Real :       ", validation_generator.classes)
print("Prediction : ", y_pred_model_reduced)
print(confusion_matrix(validation_generator.classes, y_pred_model_reduced))

Epoch 1/500
99/99 [==============================] - 4s 39ms/step - loss: 1.3859 - accuracy: 0.4967 - val_loss: 0.6963 - val_accuracy: 0.4792
Epoch 2/500
99/99 [==============================] - 4s 36ms/step - loss: 0.8046 - accuracy: 0.5090 - val_loss: 0.6924 - val_accuracy: 0.5000
Epoch 3/500
99/99 [==============================] - 4s 37ms/step - loss: 0.7466 - accuracy: 0.4995 - val_loss: 0.6977 - val_accuracy: 0.5208
Epoch 4/500
99/99 [==============================] - 4s 37ms/step - loss: 0.7521 - accuracy: 0.5379 - val_loss: 0.6935 - val_accuracy: 0.5000
Epoch 5/500
99/99 [==============================] - 4s 36ms/step - loss: 0.7223 - accuracy: 0.5059 - val_loss: 0.6923 - val_accuracy: 0.5000
Epoch 6/500
99/99 [==============================] - 4s 36ms/step - loss: 0.7036 - accuracy: 0.5186 - val_loss: 0.6937 - val_accuracy: 0.5000
Epoch 7/500
99/99 [==============================] - 4s 37ms/step - loss: 0.7175 - accuracy: 0.5375 - val_loss: 0.6973 - val_accuracy: 0.5000
Epoch 

99/99 [==============================] - 4s 36ms/step - loss: 0.4412 - accuracy: 0.7838 - val_loss: 0.7308 - val_accuracy: 0.5625
Epoch 59/500
99/99 [==============================] - 4s 36ms/step - loss: 0.3644 - accuracy: 0.8517 - val_loss: 0.7099 - val_accuracy: 0.5625
Epoch 60/500
99/99 [==============================] - 4s 36ms/step - loss: 0.6880 - accuracy: 0.7802 - val_loss: 0.7159 - val_accuracy: 0.4792
Epoch 61/500
99/99 [==============================] - 4s 36ms/step - loss: 0.4767 - accuracy: 0.7863 - val_loss: 0.7546 - val_accuracy: 0.5833
Epoch 62/500
99/99 [==============================] - 4s 37ms/step - loss: 0.8770 - accuracy: 0.7481 - val_loss: 0.8930 - val_accuracy: 0.5000
Epoch 63/500
99/99 [==============================] - 4s 36ms/step - loss: 0.3531 - accuracy: 0.8690 - val_loss: 1.1616 - val_accuracy: 0.5208
Epoch 64/500
99/99 [==============================] - 4s 36ms/step - loss: 0.4155 - accuracy: 0.8944 - val_loss: 1.7117 - val_accuracy: 0.5208
Epoch 65/500

99/99 [==============================] - 4s 37ms/step - loss: 0.0535 - accuracy: 0.9903 - val_loss: 3.3595 - val_accuracy: 0.4583
Epoch 116/500
99/99 [==============================] - 4s 36ms/step - loss: 0.3925 - accuracy: 0.9233 - val_loss: 1.6042 - val_accuracy: 0.5208
Epoch 117/500
99/99 [==============================] - 4s 36ms/step - loss: 0.1752 - accuracy: 0.9545 - val_loss: 1.7881 - val_accuracy: 0.4583
Epoch 118/500
99/99 [==============================] - 4s 36ms/step - loss: 0.0556 - accuracy: 0.9893 - val_loss: 2.0494 - val_accuracy: 0.5208
Epoch 119/500
99/99 [==============================] - 4s 36ms/step - loss: 0.0549 - accuracy: 0.9772 - val_loss: 2.3424 - val_accuracy: 0.5625
Epoch 120/500
99/99 [==============================] - 4s 37ms/step - loss: 0.1092 - accuracy: 0.9724 - val_loss: 1.1447 - val_accuracy: 0.5000
Epoch 121/500
99/99 [==============================] - 4s 36ms/step - loss: 0.1053 - accuracy: 0.9701 - val_loss: 1.5976 - val_accuracy: 0.5833
Epoch 

Epoch 172/500
99/99 [==============================] - 4s 37ms/step - loss: 0.2289 - accuracy: 0.9676 - val_loss: 2.1085 - val_accuracy: 0.4583
Epoch 173/500
99/99 [==============================] - 4s 36ms/step - loss: 0.1088 - accuracy: 0.9687 - val_loss: 2.1606 - val_accuracy: 0.5417
Epoch 174/500
99/99 [==============================] - 4s 37ms/step - loss: 0.1964 - accuracy: 0.9500 - val_loss: 2.0420 - val_accuracy: 0.5000
Epoch 175/500
99/99 [==============================] - 4s 36ms/step - loss: 0.3835 - accuracy: 0.9421 - val_loss: 2.4539 - val_accuracy: 0.5417
Epoch 176/500
99/99 [==============================] - 4s 37ms/step - loss: 0.0275 - accuracy: 0.9885 - val_loss: 2.4899 - val_accuracy: 0.5208
Epoch 177/500
99/99 [==============================] - 4s 37ms/step - loss: 0.0706 - accuracy: 0.9755 - val_loss: 2.9421 - val_accuracy: 0.5417
Epoch 178/500
99/99 [==============================] - 4s 37ms/step - loss: 0.0268 - accuracy: 0.9951 - val_loss: 2.5295 - val_accuracy:

Epoch 229/500
99/99 [==============================] - 4s 37ms/step - loss: 0.0236 - accuracy: 0.9941 - val_loss: 2.2853 - val_accuracy: 0.4375
Epoch 230/500
99/99 [==============================] - 4s 36ms/step - loss: 0.0048 - accuracy: 0.9998 - val_loss: 4.3136 - val_accuracy: 0.5833
Epoch 231/500
99/99 [==============================] - 4s 37ms/step - loss: 0.0086 - accuracy: 0.9981 - val_loss: 4.9935 - val_accuracy: 0.5833
Epoch 232/500
99/99 [==============================] - 4s 38ms/step - loss: 0.2000 - accuracy: 0.9781 - val_loss: 4.4884 - val_accuracy: 0.5208
Epoch 233/500
99/99 [==============================] - 4s 38ms/step - loss: 0.1786 - accuracy: 0.9729 - val_loss: 4.3183 - val_accuracy: 0.5208
Epoch 234/500
99/99 [==============================] - 4s 39ms/step - loss: 0.0910 - accuracy: 0.9832 - val_loss: 3.6556 - val_accuracy: 0.5625
Epoch 235/500
99/99 [==============================] - 4s 39ms/step - loss: 0.0032 - accuracy: 0.9970 - val_loss: 4.1326 - val_accuracy:

Epoch 286/500
99/99 [==============================] - 4s 37ms/step - loss: 0.0079 - accuracy: 0.9995 - val_loss: 7.0202 - val_accuracy: 0.5417
Epoch 287/500
99/99 [==============================] - 4s 39ms/step - loss: 0.1126 - accuracy: 0.9948 - val_loss: 4.3623 - val_accuracy: 0.5208
Epoch 288/500
99/99 [==============================] - 4s 37ms/step - loss: 0.0061 - accuracy: 0.9982 - val_loss: 5.0189 - val_accuracy: 0.6250
Epoch 289/500
99/99 [==============================] - 4s 38ms/step - loss: 0.4613 - accuracy: 0.9620 - val_loss: 3.4631 - val_accuracy: 0.5208
Epoch 290/500
99/99 [==============================] - 4s 36ms/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 5.1067 - val_accuracy: 0.5208
Epoch 291/500
99/99 [==============================] - 4s 38ms/step - loss: 0.0348 - accuracy: 0.9889 - val_loss: 3.6265 - val_accuracy: 0.5417
Epoch 292/500
99/99 [==============================] - 4s 37ms/step - loss: 0.0782 - accuracy: 0.9946 - val_loss: 3.9187 - val_accuracy:

99/99 [==============================] - 4s 38ms/step - loss: 5.7742e-04 - accuracy: 1.0000 - val_loss: 6.2863 - val_accuracy: 0.6042
Epoch 343/500
99/99 [==============================] - 4s 38ms/step - loss: 1.4984e-05 - accuracy: 1.0000 - val_loss: 5.4385 - val_accuracy: 0.5208
Epoch 344/500
99/99 [==============================] - 4s 38ms/step - loss: 0.0170 - accuracy: 0.9918 - val_loss: 5.5460 - val_accuracy: 0.6250
Epoch 345/500
99/99 [==============================] - 4s 37ms/step - loss: 0.0588 - accuracy: 0.9922 - val_loss: 4.2390 - val_accuracy: 0.5833
Epoch 346/500
99/99 [==============================] - 4s 37ms/step - loss: 0.0752 - accuracy: 0.9886 - val_loss: 6.0399 - val_accuracy: 0.6250
Epoch 347/500
99/99 [==============================] - 4s 38ms/step - loss: 0.0591 - accuracy: 0.9886 - val_loss: 10.2920 - val_accuracy: 0.5833
Epoch 348/500
99/99 [==============================] - 4s 37ms/step - loss: 0.2735 - accuracy: 0.9810 - val_loss: 7.7159 - val_accuracy: 0.62

Epoch 399/500
99/99 [==============================] - 4s 36ms/step - loss: 0.0064 - accuracy: 0.9989 - val_loss: 9.4159 - val_accuracy: 0.5208
Epoch 400/500
99/99 [==============================] - 4s 38ms/step - loss: 0.0129 - accuracy: 0.9974 - val_loss: 9.2929 - val_accuracy: 0.6042
Epoch 401/500
99/99 [==============================] - 4s 36ms/step - loss: 2.1302e-04 - accuracy: 1.0000 - val_loss: 8.8088 - val_accuracy: 0.6250
Epoch 402/500
99/99 [==============================] - 4s 38ms/step - loss: 0.0225 - accuracy: 0.9901 - val_loss: 6.1204 - val_accuracy: 0.6042
Epoch 403/500
99/99 [==============================] - 4s 37ms/step - loss: 0.2008 - accuracy: 0.9897 - val_loss: 4.2318 - val_accuracy: 0.5833
Epoch 404/500
99/99 [==============================] - 4s 38ms/step - loss: 0.0456 - accuracy: 0.9882 - val_loss: 4.1265 - val_accuracy: 0.5625
Epoch 405/500
99/99 [==============================] - 4s 36ms/step - loss: 0.1078 - accuracy: 0.9840 - val_loss: 4.3247 - val_accur

99/99 [==============================] - 4s 37ms/step - loss: 0.0385 - accuracy: 0.9940 - val_loss: 5.9063 - val_accuracy: 0.6042
Epoch 456/500
99/99 [==============================] - 4s 38ms/step - loss: 0.0175 - accuracy: 0.9962 - val_loss: 5.3749 - val_accuracy: 0.6042
Epoch 457/500
99/99 [==============================] - 4s 37ms/step - loss: 0.0566 - accuracy: 0.9956 - val_loss: 6.5230 - val_accuracy: 0.6042
Epoch 458/500
99/99 [==============================] - 4s 38ms/step - loss: 3.1641e-06 - accuracy: 1.0000 - val_loss: 7.0183 - val_accuracy: 0.6042
Epoch 459/500
99/99 [==============================] - 4s 37ms/step - loss: 1.4220e-04 - accuracy: 1.0000 - val_loss: 6.0169 - val_accuracy: 0.6250
Epoch 460/500
99/99 [==============================] - 4s 38ms/step - loss: 0.0190 - accuracy: 0.9970 - val_loss: 9.4631 - val_accuracy: 0.6042
Epoch 461/500
99/99 [==============================] - 4s 37ms/step - loss: 0.0805 - accuracy: 0.9914 - val_loss: 7.6494 - val_accuracy: 0.625

In [ ]:
bigger_model = Sequential()
#bigger_model.add.Conv2D(64, 
bigger_model.add(Flatten(input_shape = (img_rows, img_cols, n_channels))),
bigger_model.add(Dense(256)),
bigger_model.add(Activation('relu'))
bigger_model.add(Dropout(0.25)),
bigger_model.add(Dense(128)),
bigger_model.add(Activation('relu'))
bigger_model.add(Dropout(0.25)),
bigger_model.add(Dense(64)),
bigger_model.add(Activation('relu'))
bigger_model.add(Dropout(0.5)),
bigger_model.add(Dense(16)),
bigger_model.add(Activation('relu'))
bigger_model.add(Dense(2, activation=tf.nn.softmax))
bigger_model.add(Activation('sigmoid'))
    

bigger_model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

bigger_model_history = bigger_model.fit(train_generator,
                                        steps_per_epoch=num_of_train_samples // batch_size,
                                        epochs=epochs,
                                        validation_data=validation_generator,
                                        validation_steps=num_of_test_samples // batch_size,
                                        #callbacks=[monitor],verbose=2 # interrompe quando o overfiting começa
                                        )

Y_pred_bigger_model = bigger_model.predict(validation_generator, num_of_test_samples // batch_size)
y_pred_bigger_model = np.argmax(Y_pred_bigger_model, axis=1)
print("Real :       ", validation_generator.classes)
print("Prediction : ", y_pred_bigger_model)
print('Confusion ValueError: could not broadcast input array from shape (1,100,100) into shape (100,100,3)Matrix')
print(confusion_matrix(validation_generator.classes, y_pred_bigger_model))

Epoch 1/500
99/99 [==============================] - 7s 65ms/step - loss: 0.7285 - accuracy: 0.5739 - val_loss: 0.8133 - val_accuracy: 0.5000
Epoch 2/500
99/99 [==============================] - 6s 60ms/step - loss: 0.7622 - accuracy: 0.5514 - val_loss: 0.8341 - val_accuracy: 0.4792
Epoch 3/500
99/99 [==============================] - 6s 62ms/step - loss: 0.8097 - accuracy: 0.4922 - val_loss: 0.8341 - val_accuracy: 0.4792
Epoch 4/500
99/99 [==============================] - 6s 60ms/step - loss: 0.7687 - accuracy: 0.5413 - val_loss: 0.8341 - val_accuracy: 0.4792
Epoch 5/500
99/99 [==============================] - 6s 61ms/step - loss: 0.8038 - accuracy: 0.5105 - val_loss: 0.7924 - val_accuracy: 0.5208
Epoch 6/500
99/99 [==============================] - 6s 59ms/step - loss: 0.8033 - accuracy: 0.5092 - val_loss: 0.7924 - val_accuracy: 0.5208
Epoch 7/500
99/99 [==============================] - 6s 58ms/step - loss: 0.7977 - accuracy: 0.5159 - val_loss: 0.8133 - val_accuracy: 0.5000
Epoch 

99/99 [==============================] - 6s 56ms/step - loss: 0.7957 - accuracy: 0.5175 - val_loss: 0.7924 - val_accuracy: 0.5208
Epoch 59/500
99/99 [==============================] - 6s 57ms/step - loss: 0.8683 - accuracy: 0.4449 - val_loss: 0.7924 - val_accuracy: 0.5208
Epoch 60/500
99/99 [==============================] - 6s 58ms/step - loss: 0.7966 - accuracy: 0.5166 - val_loss: 0.7924 - val_accuracy: 0.5208
Epoch 61/500
99/99 [==============================] - 6s 58ms/step - loss: 0.7904 - accuracy: 0.5228 - val_loss: 0.8133 - val_accuracy: 0.5000
Epoch 62/500
99/99 [==============================] - 6s 60ms/step - loss: 0.8279 - accuracy: 0.4856 - val_loss: 0.8133 - val_accuracy: 0.5000
Epoch 63/500
99/99 [==============================] - 6s 61ms/step - loss: 0.7691 - accuracy: 0.5442 - val_loss: 0.8133 - val_accuracy: 0.5000
Epoch 64/500
99/99 [==============================] - 6s 57ms/step - loss: 0.8082 - accuracy: 0.5051 - val_loss: 0.7924 - val_accuracy: 0.5208
Epoch 65/500

99/99 [==============================] - 5s 51ms/step - loss: 0.8117 - accuracy: 0.5016 - val_loss: 0.7924 - val_accuracy: 0.5208
Epoch 116/500
99/99 [==============================] - 5s 55ms/step - loss: 0.8079 - accuracy: 0.5053 - val_loss: 0.8133 - val_accuracy: 0.5000
Epoch 117/500
99/99 [==============================] - 5s 53ms/step - loss: 0.8108 - accuracy: 0.5025 - val_loss: 0.8133 - val_accuracy: 0.5000
Epoch 118/500
99/99 [==============================] - 6s 63ms/step - loss: 0.7687 - accuracy: 0.5445 - val_loss: 0.8133 - val_accuracy: 0.5000
Epoch 119/500
99/99 [==============================] - 6s 58ms/step - loss: 0.8085 - accuracy: 0.5048 - val_loss: 0.7924 - val_accuracy: 0.5208
Epoch 120/500
99/99 [==============================] - 6s 62ms/step - loss: 0.8456 - accuracy: 0.4677 - val_loss: 0.7924 - val_accuracy: 0.5208
Epoch 121/500
99/99 [==============================] - 6s 58ms/step - loss: 0.8398 - accuracy: 0.4735 - val_loss: 0.7924 - val_accuracy: 0.5208
Epoch 

Epoch 172/500
99/99 [==============================] - 5s 54ms/step - loss: 0.8055 - accuracy: 0.5078 - val_loss: 0.7924 - val_accuracy: 0.5208
Epoch 173/500
99/99 [==============================] - 6s 64ms/step - loss: 0.8354 - accuracy: 0.4779 - val_loss: 0.7924 - val_accuracy: 0.5208
Epoch 174/500
99/99 [==============================] - 6s 59ms/step - loss: 0.8303 - accuracy: 0.4829 - val_loss: 0.8133 - val_accuracy: 0.5000
Epoch 175/500
99/99 [==============================] - 6s 58ms/step - loss: 0.7960 - accuracy: 0.5173 - val_loss: 0.7924 - val_accuracy: 0.5208
Epoch 176/500
99/99 [==============================] - 5s 54ms/step - loss: 0.7650 - accuracy: 0.5483 - val_loss: 0.8133 - val_accuracy: 0.5000
Epoch 177/500
99/99 [==============================] - 7s 73ms/step - loss: 0.7942 - accuracy: 0.5190 - val_loss: 0.7924 - val_accuracy: 0.5208
Epoch 178/500
99/99 [==============================] - 8s 83ms/step - loss: 0.8156 - accuracy: 0.4976 - val_loss: 0.7924 - val_accuracy:

In [ ]:
baseline_model = Sequential()
baseline_model.add(Flatten(input_shape = (img_rows, img_cols, n_channels))),
baseline_model.add(Dense(128)),
baseline_model.add(Activation('relu'))
baseline_model.add(Dropout(0.25)),
baseline_model.add(Dense(16)),
baseline_model.add(Activation('relu'))
baseline_model.add(Dense(2))
baseline_model.add(Activation('sigmoid'))


baseline_model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

baseline_model_history = baseline_model.fit(train_generator,
                                        steps_per_epoch=num_of_train_samples // batch_size,
                                        epochs=epochs,
                                        validation_data=validation_generator,
                                        validation_steps=num_of_test_samples // batch_size,
                                        #callbacks=[monitor],verbose=2 # interrompe quando o overfiting começa
                                        )

Y_pred_baseline_model = baseline_model.predict(validation_generator, num_of_test_samples // batch_size)
y_pred_baseline_model = np.argmax(Y_pred_baseline_model, axis=1)
print("Real :       ", validation_generator.classes)
print("Prediction : ", y_pred_baseline_model)
print(confusion_matrix(validation_generator.classes, y_pred_baseline_model))

In [ ]:
bigger_model1 = keras.models.Sequential()
bigger_model1.add(Flatten(input_shape = (img_rows, img_cols, n_channels))),
bigger_model1.add(Dense(128, activation=tf.nn.relu)),
bigger_model1.add(Activation('relu'))
bigger_model1.add(Dropout(0.25))
bigger_model1.add(Dense(64))
bigger_model1.add(Activation('relu'))
bigger_model1.add(Dropout(0.5))
bigger_model1.add(Dense(16))
bigger_model1.add(Activation('relu'))
bigger_model1.add(Dense(2))
bigger_model1.add(Activation('sigmoid'))

bigger_model1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

bigger_model1_history = bigger_model1.fit(train_generator,
                                        steps_per_epoch=num_of_train_samples // batch_size,
                                        epochs=epochs,
                                        validation_data=validation_generator,
                                        validation_steps=num_of_test_samples // batch_size,
                                        #callbacks=[monitor],verbose=2 # interrompe quando o overfiting começa
                                        )

Y_pred_bigger_model1 = bigger_model1.predict(validation_generator, num_of_test_samples // batch_size)
y_pred_baseline_model = np.argmax(Y_pred_baseline_model, axis=1)
print("Real :       ", validation_generator.classes)
print("Prediction : ", y_pred_baseline_model)
print(confusion_matrix(validation_generator.classes, y_pred_baseline_model))

In [ ]:
smaller_model1 = keras.models.Sequential()
smaller_model1.add(Flatten(input_shape = (img_rows, img_cols, n_channels)))
smaller_model1.add(Dense(64))
smaller_model1.add(Activation('relu'))
smaller_model1.add(Dense(2))
smaller_model1.add(Activation('sigmoid'))

smaller_model1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

smaller_model1_history = smaller_model1.fit(train_generator,
                                            steps_per_epoch=num_of_train_samples // batch_size,
                                            epochs=epochs,
                                            validation_data=validation_generator,
                                            validation_steps=num_of_test_samples // batch_size,
                                            #callbacks=[monitor],verbose=2 # interrompe quando o overfiting começa
                                            )
                   
Y_pred_smaller_model1 = smaller_model1.predict(validation_generator, num_of_test_samples // batch_size)
y_pred_smaller_model1 = np.argmax(Y_pred_smaller_model1, axis=1)
#print(Y_pred)
print("Real :       ", validation_generator.classes)
print("Prediction : ", y_pred_smaller_model1)
print(confusion_matrix(validation_generator.classes, y_pred_smaller_model1))

In [ ]:
Y_pred = model.predict(validation_generator, num_of_test_samples // batch_size)
y_pred = np.argmax(Y_pred, axis=1)
print("Real :       ", validation_generator.classes)
print("Prediction : ", y_pred)
print('Confusion ValueError: could not broadcast input array from shape (1,100,100) into shape (100,100,3)Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))

In [ ]:
print('Classification Report')
target_names = ['daninhas', 'sem_daninhas']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

In [ ]:
def map_accuracy(*names, key='accuracy'):
    '''
    names: conjunto de nomes de histories gerados a partir do model.fit(), cada
    nome adicionado fará com que um modelo a mais seja considerado no mapamento 
    geral de acuracias:
    
    description: É feito para analisar um conjunto grande de modelos treinados,
    os histories aqui gerados irão ser comparados, inserindo variás curvas no 
    mesmo gráfico
    
    return: tupla de arrays
    retorna um array de histories em conjunto com um array de resultados de 
    acurácia. O indx do melhor resutado de acurácia é armazenado e salvo para
    plotagem individual
    '''
    _count = 0
    result_value = []
    plt.figure(figsize=(16,10))
    for history_name in names:
        result_value.append(history_name.history['accuracy'][-1])
        _count += 1
        key='accuracy'
        val = plt.plot(history.epoch, history_name.history['val_'+key])
        plt.plot(history.epoch, history_name.history[key], color=val[0].get_color(), label=' Train')
        plt.xlabel('Epochs')
        plt.ylabel(key.replace('_',' ').title())
        plt.legend()
        plt.xlim([0, max(history.epoch)])
        plt.ylim([0, 1])

    return result_value, names 

values, histories = map_accuracy(history, history_model_reduced, bigger_model_history, baseline_model_history, bigger_model1_history, smaller_model1_history)

In [ ]:
for i in range(0, len(values), 1):
    if values[i] == max(values):
        index = i 
key='accuracy'
print(index)
print(histories[index].history['accuracy'][-1])
plt.plot(histories[index].history['accuracy'])
plt.plot(histories[index].history['val_accuracy'])
plt.title('model accuracy of best model')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss of best model')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
print(history.history['accuracy'][-1])
    
result = [history.history['accuracy'][-1], bigger_model_history.history['accuracy'][-1], 
            baseline_model_history.history['accuracy'][-1], bigger_model1_history.history['accuracy'][-1],
            smaller_model1_history.history['accuracy'][-1]]

print(result[1])

In [ ]:
def plot_history(histories, key='accuracy'):
    plt.figure(figsize=(16,10))
    
    for name, history in histories:
        val = plt.plot(history.epoch, history.history['val_'+key],'--', label=name.title()+' Val')
        plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
                label=name.title()+' Train')

        plt.xlabel('Epochs')
        plt.ylabel(key.replace('_',' ').title())
        plt.legend()
        plt.xlim([0, max(history.epoch)])
        plt.ylim([0, 1])

In [ ]:
plot_history([
             ('model', history),
             ('history_model_reduced', history_model_reduced),
             ('baseline', baseline_model_history),
             ('bigger_model1', bigger_model_history)
             #('vgg', vgg_style_model_history)
             ])

In [ ]:
print(history.history['accuracy'][-1])
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy of best model')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss of best model')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
image_path = '../../sensix_daninhas/dataset_100x100/validation/daninha'

images = []

for img in os.listdir(image_path):
        img = os.path.join(image_path, img)
        img = image.load_img(img, target_size=(100,100))
        img = image.img_to_array(img)
        #print(img)
        img = np.expand_dims(img, axis=0)
        processed_image = np.array(img, dtype="float")
        images.append(processed_image)
        
images = np.vstack(images)
print(len(images))

In [ ]:
images_data = []
filenames = []
count_daninha = 0
count_sem_daninha = 0
count = 0
pred_result = model.predict(validation_generator, len(images) // batch_size+1)
#print(pred_result)
plt.figure(figsize=(20,20))
grid_size = min(0, len(pred_result))

for filename in os.listdir(image_path):
    images_data.append(pred_result)
    filenames.append(filename)
    plt.subplot(5, 5, count+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(images[count].astype(np.int32), cmap=plt.cm.binary)
    if pred_result[count][0]> pred_result[count][1]:
        legend_x = plt.xlabel("daniha")
        legend_x.set_color("red")
        count_daninha = count_daninha + 1
        if count >= num_of_test_samples -1:
            break
    else :
        legend_x = plt.xlabel("sem_daninha")
        legend_x.set_color("green")
        count_sem_daninha = count_sem_daninha + 1
    count += 1

print("count_daninha", count_daninha)
print("count_sem_daninha", count_sem_daninha)
plt.suptitle('Classifição Daninhas', y = 0.92, fontsize=24)
plt.show(images_data)